In [1]:
import numpy as np
import pandas as pd 
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\anaya\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df = pd.read_csv(r'C:\Users\anaya\OneDrive\Desktop\Anay Masters\Intro to NLP\NLP_Group_7\FullyProcessedDataset.csv')

tokenized_corpus = [text.split() for text in df['Lemmatized_Lyrics']]

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=35, random_state = 42)

song_topic_distribution = [lda_model[doc] for doc in corpus]

In [156]:

user_input = input("Enter song index: ")
user_song = (df.index[df['SName'] == user_input]).item()
print(user_song)
df.head(5)


55


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\r\nThis beautiful...,en,come take beautiful bruise color everything fa...,0.8316,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.431830
1,1,46,139365,/foo-fighters/,Baker Street,/foo-fighters/baker-street.html,Winding you way down on baker street\r\nLight ...,en,winding way baker street light head dead foot ...,-0.1561,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.544712
2,2,315,139477,/foo-fighters/,Savior Breath,/foo-fighters/savior-breath.html,[Verse 1]\r\nBreathe oh breathe now smother me...,en,verse breathe oh breathe smother clever onto o...,0.0854,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.684712
3,3,319,139496,/foo-fighters/,The Line,/foo-fighters/the-line.html,Yes or no?\r\nWhat is truth\r\nBut a dirty bla...,en,yes truth dirty black cloud come blue wrong ri...,-0.9337,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.520837
4,4,835,139485,/foo-fighters/,Statues,/foo-fighters/statues.html,You and I were two old and tortured souls\r\nR...,en,two old torture soul repair love break fang li...,0.0258,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.551659


In [157]:
def calculate_hellinger_distance(song_dist_1, song_dist_2):
    return hellinger(song_dist_1, song_dist_2)

input_song_index = user_song

input_song_dist = song_topic_distribution[input_song_index]

hellinger_distances = []

for i, song_dist in enumerate(song_topic_distribution):

    distance = calculate_hellinger_distance(input_song_dist, song_dist)
    hellinger_distances.append((distance))

df['HDF'] = hellinger_distances
df.head(60)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\r\nThis beautiful...,en,come take beautiful bruise color everything fa...,0.8316,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.431830
1,1,46,139365,/foo-fighters/,Baker Street,/foo-fighters/baker-street.html,Winding you way down on baker street\r\nLight ...,en,winding way baker street light head dead foot ...,-0.1561,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.544712
2,2,315,139477,/foo-fighters/,Savior Breath,/foo-fighters/savior-breath.html,[Verse 1]\r\nBreathe oh breathe now smother me...,en,verse breathe oh breathe smother clever onto o...,0.0854,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.684712
3,3,319,139496,/foo-fighters/,The Line,/foo-fighters/the-line.html,Yes or no?\r\nWhat is truth\r\nBut a dirty bla...,en,yes truth dirty black cloud come blue wrong ri...,-0.9337,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.520837
4,4,835,139485,/foo-fighters/,Statues,/foo-fighters/statues.html,You and I were two old and tortured souls\r\nR...,en,two old torture soul repair love break fang li...,0.0258,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.551659
5,5,1837,139500,/foo-fighters/,This Will Be Our Year,/foo-fighters/this-will-be-our-year.html,The warmth of your love's\r\nLike the warmth f...,en,warmth love like warmth sun year take long tim...,0.9870,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.743344
6,6,2478,139457,/foo-fighters/,No Way Back,/foo-fighters/no-way-back.html,"Lately, I've been\r\nLivin' in my head\r\nThe ...",en,lately livin head rest dead die truth make bel...,0.9834,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.502175
7,7,2800,139494,/foo-fighters/,The Feast and The Famine,/foo-fighters/the-feast-and-the-famine.html,That night they were burning of the truth\r\nD...,en,night burn truth corner th u take soul take fo...,-0.8419,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.631326
8,8,4356,139407,/foo-fighters/,Gimme Stitches,/foo-fighters/gimme-stitches.html,I can be your right of way\r\nSo we can get ou...,en,right way get always one run everyone cause ev...,-0.8420,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.655936
9,9,5613,139454,/foo-fighters/,Never Talking To You Again,/foo-fighters/never-talking-to-you-again-2.html,There are things that I'd like to say\r\nBut I...,en,thing like say never talk thing like phrase wa...,0.6846,Foo Fighters,Rock Alternativo; Rock; Hard Rock,0.515896


In [158]:
userGenre = df['Genres'].iloc[user_song]
genreReducedDf = df.loc[df['Genres'] == userGenre]
print(df.shape[0])
print(genreReducedDf.shape[0])
genreReducedDf.head(5)

38276
275


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF
29,29,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839,MxPx,Punk Rock,0.483795
30,30,1164,290828,/mxpx/,Lucky Guy,/mxpx/lucky-guy.html,I'm not a millionaire\r\nBut she still loves m...,en,millionaire still loves know care still loves ...,0.9937,MxPx,Punk Rock,0.484915
31,31,1263,290800,/mxpx/,I Can Be Friends With You,/mxpx/i-can-be-friends-with-you.html,[Originally by Petra]\r\n\r\nI can be friends ...,en,originally petra friend well friend man know f...,0.9982,MxPx,Punk Rock,0.529709
32,32,1493,290785,/mxpx/,Foolish,/mxpx/foolish.html,"Some people say that I threw my brain away,\r\...",en,people say throw brain away illogical much say...,0.1571,MxPx,Punk Rock,0.273645
33,33,2558,290956,/mxpx/,Your problem my emergency,/mxpx/your-problem-my-emergency.html,"Get a clue, I'm not here for you\r\nAnd I'm no...",en,get clue like go to say right cause end like g...,-0.4767,MxPx,Punk Rock,0.457430


In [159]:
indexList = genreReducedDf['Unnamed: 0.2'].tolist()
newTopIndex = indexList[0]
print(newTopIndex)
userSongIndexUpdated = user_song - newTopIndex
print(userSongIndexUpdated)
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(lyrics):
    sentiment = analyzer.polarity_scores(lyrics)
    sentiment_score = sentiment['compound']
    return sentiment_score

sentiment = get_sentiment_score(genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated])
print(sentiment)

SSF_list = []
for i in range(genreReducedDf.shape[0]):
    SSF_list.append(abs(genreReducedDf['Sentiment'].iloc[i] - sentiment))

genreReducedDf['SSF'] = SSF_list
genreReducedDf.head(56)


29
26
0.296


C:\Users\anaya\AppData\Local\Temp\ipykernel_28008\2633366284.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['SSF'] = SSF_list


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF
29,29,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839,MxPx,Punk Rock,0.483795,0.5879
30,30,1164,290828,/mxpx/,Lucky Guy,/mxpx/lucky-guy.html,I'm not a millionaire\r\nBut she still loves m...,en,millionaire still loves know care still loves ...,0.9937,MxPx,Punk Rock,0.484915,0.6977
31,31,1263,290800,/mxpx/,I Can Be Friends With You,/mxpx/i-can-be-friends-with-you.html,[Originally by Petra]\r\n\r\nI can be friends ...,en,originally petra friend well friend man know f...,0.9982,MxPx,Punk Rock,0.529709,0.7022
32,32,1493,290785,/mxpx/,Foolish,/mxpx/foolish.html,"Some people say that I threw my brain away,\r\...",en,people say throw brain away illogical much say...,0.1571,MxPx,Punk Rock,0.273645,0.1389
33,33,2558,290956,/mxpx/,Your problem my emergency,/mxpx/your-problem-my-emergency.html,"Get a clue, I'm not here for you\r\nAnd I'm no...",en,get clue like go to say right cause end like g...,-0.4767,MxPx,Punk Rock,0.457430,0.7727
34,34,2640,290709,/mxpx/,"Scooby Doo, Where Are You?",/mxpx/scooby-doo-where-are-you.html,"Scooby Dooby Doo,\r\nWhere are you?\r\nWe got ...",en,scooby dooby doo get work scooby dooby doo nee...,0.7096,MxPx,Punk Rock,0.793978,0.4136
35,35,4745,290908,/mxpx/,The Hoo-ha Jangle,/mxpx/the-hoo-ha-jangle.html,Please stop all the talking cause we've heard ...,en,please stop talk cause hear buying sell althou...,0.1650,MxPx,Punk Rock,0.592969,0.1310
36,36,6300,290802,/mxpx/,I Would Walk 500 Miles,/mxpx/i-would-walk-500-miles.html,"When I wake up, yeah I know Im gonna be,\r\nI'...",en,wake yeah know I m go to go to man wake next g...,-0.5886,MxPx,Punk Rock,0.799646,0.8846
37,37,6965,290768,/mxpx/,Dust In The Wind,/mxpx/dust-in-the-wind.html,I close my eyes\r\nOnly for a moment\r\nAnd th...,en,close eye moment moment go dream pass eye curi...,-0.3182,MxPx,Punk Rock,0.789146,0.6142
38,38,7387,290715,/mxpx/,17 (I Saw Her Standing There),/mxpx/17-i-saw-her-standing-there.html,Well she was just 17\r\nYou know what I mean\r...,en,well know mean way look way beyond compare cou...,0.9509,MxPx,Punk Rock,0.407437,0.6549


In [160]:
tfidf_vectorizer = TfidfVectorizer()
CSF_list = []
for i in range(genreReducedDf.shape[0]):
    lyrics_matrix = tfidf_vectorizer.fit_transform([genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated], genreReducedDf['Lemmatized_Lyrics'].iloc[i]])
    lyrics_similarity = cosine_similarity(lyrics_matrix)
    CSF_list.append(1 - (lyrics_similarity[0][1]))
genreReducedDf['CSF'] = CSF_list
genreReducedDf.head(30)


C:\Users\anaya\AppData\Local\Temp\ipykernel_28008\4020052563.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['CSF'] = CSF_list


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF,CSF
29,29,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839,MxPx,Punk Rock,0.483795,0.5879,9.861369e-01
30,30,1164,290828,/mxpx/,Lucky Guy,/mxpx/lucky-guy.html,I'm not a millionaire\r\nBut she still loves m...,en,millionaire still loves know care still loves ...,0.9937,MxPx,Punk Rock,0.484915,0.6977,9.315417e-01
31,31,1263,290800,/mxpx/,I Can Be Friends With You,/mxpx/i-can-be-friends-with-you.html,[Originally by Petra]\r\n\r\nI can be friends ...,en,originally petra friend well friend man know f...,0.9982,MxPx,Punk Rock,0.529709,0.7022,9.892020e-01
32,32,1493,290785,/mxpx/,Foolish,/mxpx/foolish.html,"Some people say that I threw my brain away,\r\...",en,people say throw brain away illogical much say...,0.1571,MxPx,Punk Rock,0.273645,0.1389,8.539865e-01
33,33,2558,290956,/mxpx/,Your problem my emergency,/mxpx/your-problem-my-emergency.html,"Get a clue, I'm not here for you\r\nAnd I'm no...",en,get clue like go to say right cause end like g...,-0.4767,MxPx,Punk Rock,0.457430,0.7727,8.942024e-01
34,34,2640,290709,/mxpx/,"Scooby Doo, Where Are You?",/mxpx/scooby-doo-where-are-you.html,"Scooby Dooby Doo,\r\nWhere are you?\r\nWe got ...",en,scooby dooby doo get work scooby dooby doo nee...,0.7096,MxPx,Punk Rock,0.793978,0.4136,9.671692e-01
35,35,4745,290908,/mxpx/,The Hoo-ha Jangle,/mxpx/the-hoo-ha-jangle.html,Please stop all the talking cause we've heard ...,en,please stop talk cause hear buying sell althou...,0.1650,MxPx,Punk Rock,0.592969,0.1310,9.878655e-01
36,36,6300,290802,/mxpx/,I Would Walk 500 Miles,/mxpx/i-would-walk-500-miles.html,"When I wake up, yeah I know Im gonna be,\r\nI'...",en,wake yeah know I m go to go to man wake next g...,-0.5886,MxPx,Punk Rock,0.799646,0.8846,9.156652e-01
37,37,6965,290768,/mxpx/,Dust In The Wind,/mxpx/dust-in-the-wind.html,I close my eyes\r\nOnly for a moment\r\nAnd th...,en,close eye moment moment go dream pass eye curi...,-0.3182,MxPx,Punk Rock,0.789146,0.6142,9.380400e-01
38,38,7387,290715,/mxpx/,17 (I Saw Her Standing There),/mxpx/17-i-saw-her-standing-there.html,Well she was just 17\r\nYou know what I mean\r...,en,well know mean way look way beyond compare cou...,0.9509,MxPx,Punk Rock,0.407437,0.6549,8.765940e-01


In [161]:
# for i in range(df.shape[0]):
#     if df['Cosine_Similarity'].iloc[i] >= 0.3:
#         print(df['Cosine_Similarity'].iloc[i])

genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']
genreReducedDf = genreReducedDf.nsmallest(6,["SF"])
genreReducedDf.drop(user_song, axis = 0, inplace = True)
genreReducedDf

C:\Users\anaya\AppData\Local\Temp\ipykernel_28008\2616333906.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Artist,Genres,HDF,SSF,CSF,SF
44,44,14063,290872,/mxpx/,Rock N' Roll Girl,/mxpx/rock-n-roll-girl.html,"I don't know what to do, I dont know what to s...",en,know do not know say hurt let go look way oh s...,0.2444,MxPx,Punk Rock,0.296365,0.0516,0.871784,1.219749
32,32,1493,290785,/mxpx/,Foolish,/mxpx/foolish.html,"Some people say that I threw my brain away,\r\...",en,people say throw brain away illogical much say...,0.1571,MxPx,Punk Rock,0.273645,0.1389,0.853987,1.266532
4792,4792,19214,292416,/good-riddance/,Blueliner,/good-riddance/blueliner.html,Been set up to take the fall\r\nTried hard not...,en,set take fall try hard lose shoot straight rig...,0.2500,Good Riddance,Punk Rock,0.358261,0.0460,0.875475,1.279736
40,40,10210,290936,/mxpx/,Under Lock And Key,/mxpx/under-lock-and-key.html,"theres something crazy, something strange abou...",en,there s something crazy something strange way ...,0.6096,MxPx,Punk Rock,0.206908,0.3136,0.841764,1.362272
75,75,32642,290769,/mxpx/,Ears To Hear,/mxpx/ears-to-hear.html,right now there's\r\nnothing left for me to\r\...,en,right nothing leave say want go away hear one ...,0.1037,MxPx,Punk Rock,0.375407,0.1923,0.839150,1.406857


In [162]:
RecommendationDF = genreReducedDf[['Artist', 'SName']]
RecommendationDF.reset_index(inplace = True)
RecommendationDF = RecommendationDF.drop('index', axis = 1)
RecommendationDF

,Artist,SName
0,MxPx,Rock N' Roll Girl
1,MxPx,Foolish
2,Good Riddance,Blueliner
3,MxPx,Under Lock And Key
4,MxPx,Ears To Hear
